In [ ]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

from fnmatch import fnmatch

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pattern import web


# set some nicer defaults for matplotlib
from matplotlib import rcParams


# Mathjobs job page
url = "https://www.mathjobs.org/jobs?joblist-0-3----t"

job_page = requests.get(url).text.encode('ascii', 'ignore')

def is_jobs(l):
    """return True if this content include a job code"""    
    pattern = '*[*'
    return fnmatch(l, pattern)

def extractalljobs(html):
    dom = web.Element(html)
#    result_headers=['Index', 'Code','Institute','Title','url']
    
    Index = []
    Code = []
    Inst = []
    Title = []
    URL = []
    
    location = []
    subject = []
    deadline = []
    
    i = 0
    
    for listing in dom.by_tag('dl'):

        Index.append(i)
        for jobs in listing.by_tag('li'):
            job_content=jobs.content.encode('ascii','ignore')
            
            if is_jobs(job_content):
                i = i + 1
                Code.append(jobs.by_tag('a')[0].content.encode('ascii','ignore'))
                Inst.append(listing.by_tag('b')[0].by_tag('a')[0].content.encode('ascii','ignore')) #remove the 'u in school name
                m = re.search('] (.+?)&nbsp', job_content)
                if m:
                    Title.append(m.group(1).replace('<i>','').replace('</i>',''))                     
                URL.append(jobs.by_tag('a')[0].attributes.get('href', '')) 
                
    URL = ["http://www.mathjobs.org"+l for l in URL]   
    URL = [x.encode('UTF8') for x in URL]
    
    for url in URL:
        jobinfo=extractsinglejob(url)
        location.append(jobinfo['location'])
        subject.append(jobinfo['subject'])
        deadline.append(jobinfo['deadline'])
                        
    result = pd.DataFrame({'Index': range(1,i+1), 'Code': Code, 'Institution': Inst, 'Title': Title, 
                           'URL': URL, 'Location':location, 'Subject': subject, 'Deadline': deadline})
    return result


# The following routine read a url of a specific job and extract the research field, location, deadline, etc. 
def extractsinglejob(url):
    myjob = requests.get(url).text.encode('ascii', 'ignore')
    dom = web.Element(myjob)
    
    m = re.search('Position Location(.+)<a', str(dom))
    if m:
        location = m.group(1).replace(':</b> ','').replace(' [','')
    else:
        location = 'nan'            
        
    m = re.search('Subject Area(.+)<br', str(dom))
    if m:
        subject = m.group(1).replace(':</b> ','').replace(' [','')
    else:
        subject = 'nan'
    
    m = re.search('Application Deadline(.+)<br', str(dom))
    if m:
        deadline = m.group(1).replace(':</b> ','').replace(' [','')
    else:
        deadline = 'nan'

    result = {'location': location, 'subject': subject, 'deadline': deadline}
    
    return result
    
DF = extractalljobs(job_page)
NewDeadline = [DF.Deadline[j].split('  (')[0] for j in DF.index]
DF.Deadline = NewDeadline
DF_sorted=DF.sort("Deadline") # Sort jobs by deadline

In [1]:
from sqlalchemy.types import String
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:')

DF.to_sql('mathjoblisting',engine,index=False)

qq ='''SELECT Code,Deadline,Institution,Title FROM mathjoblisting 
Where Deadline BETWEEN 2015 AND 2017 ORDER BY Deadline ASC'''

pd.read_sql_query(qq, engine) # Show jobs under certain condition using SQL engine

,Code,Deadline,Institution,Title
0,PDF,2015/07/21,University of Newcastle,"Algebra, Post Doctoral Fellow"
1,CDSEPD,2015/10/15,"Mathematics, University at Buffalo, SUNY",CDSE Postdoc
2,POSTDOCPOSITIONS,2015/11/01,"Hausdorff Center for Mathematics, University o...",Postdoctoral Positions
3,PDF2015,2015/11/01,"Institute for Quantum Computing, University of...","Quantum Information Processing, Postdoctoral F..."
4,CARRIERFELLOW,2015/11/15,"School of Engineering and Applied Science, Har...","Applied Mathematics, The George F. Carrier* Po..."
5,NEUKFELLOW1616,2015/11/15,Neukom Institute for Computational Science,"Computational Science, Dartmouth College Neuko..."
6,PDFRF,2015/11/15,"Mathematical Sciences Institute, The Australia...",Postdoctoral Fellow / Research Fellow
7,PDF16,2015/11/15,"Department of Mathematics, University of Briti...",Postdoctoral Fellowships
8,PIMSPDF16,2015/11/15,"Department of Mathematics, University of Briti...",UBC-PIMS Distinguished Postdoctoral Fellowship
9,SLIM15,2015/11/15,"Department of Mathematics, University of Briti...","Seismic data analysis, Postdoctoral fellows (C..."


In [2]:
DF.head()

,Code,Deadline,Index,Institution,Location,Subject,Title,URL
0,POSTDOC,2015/11/30,1,School of Mathematical and Statistical Science...,"Tempe, Arizona 85287-1804, United States",all areas in mathematical and statistical scie...,all areas in mathematical and statistical scie...,http://www.mathjobs.org/jobs/jobs/7645
1,RTGPOSTDOC,2015/11/30,2,School of Mathematical and Statistical Science...,"Tempe, Arizona 85287-1804, United States",Applied Mathematics and Statistics,"Applied Mathematics and Statistics, Postdoctor...",http://www.mathjobs.org/jobs/jobs/7644
2,POSTDOC,,3,"Mathematics and Statistics, Auburn University","Auburn University, Alabama 36849, United States",sMathematics and/or Statistics,"Mathematics and/or Statistics, PostDoctoral Fe...",http://www.mathjobs.org/jobs/jobs/7923
3,POSTDOC,2015/12/01,4,"Department of Mathematics, Baylor University","Waco, Texas Texas, United States",areas compatible with current faculty,"areas compatible with current faculty, Post Do...",http://www.mathjobs.org/jobs/jobs/7682
4,POSTDOC,2015/11/24,5,Beijing Computational Science Research Center,"Beijing, Beijing 100084, China",computational science,"computational science, Postdoc Positions (5+) ...",http://www.mathjobs.org/jobs/jobs/6779
